In [1]:
import torch

# پارامترهای ساده
T, B, N = 2, 1, 3                # توالی تاریخی ۲، یک بَچ، ۳ گره
D, F = 5, 1                      # ابعاد ورودی = ۵، ابعاد خروجی = ۱
hidden_dim, heads = 4, 2         # H=4 (باید قابل تقسیم بر heads باشد)
dist_adj = torch.eye(N)          # گراف فاصله ساده: ماتریس همانی

# دوتا تست: هم بدون DAGMA و هم با DAGMA دستی
x_with_pre = torch.randn(T+1, B, N, D)  # شکل (3,1,3,5)
print("x_with_pre.shape:", x_with_pre.shape)


x_with_pre.shape: torch.Size([3, 1, 3, 5])


In [2]:
from nns.case.dag_gen_gru import DagGenGRUg4s2v1

# مدل را با use_dagma=False بساز
model_nd = DagGenGRUg4s2v1(
    num_nodes=N, in_feats_dim=D, out_feats_dim=F,
    hidden_dim=hidden_dim, num_layers=1, num_heads=heads,
    feats_layers=1, dist_adj=dist_adj, agg_feats='ori',
    node_norm=False, use_norm=False,
    use_dagma=False, use_lib_dagma=False  # بدون هیچ پروجکتوری
)

# فراموش نکن اگر در GPU هستی model_nd.to(device)
model_nd.eval()
with torch.no_grad():
    graphs_nd, reconst_nd = model_nd(x_with_pre, gen_graph_only=False)

print("Without DAGMA → graphs:", graphs_nd.shape)
print("Without DAGMA → reconst:", reconst_nd.shape)


Without DAGMA → graphs: torch.Size([2, 1, 2, 3, 3])
Without DAGMA → reconst: torch.Size([2, 1, 3, 1])


In [3]:
from nns.case.dag_gen_gru import DagGenGRUg4s2v1

# مدل با پروجکتور دستی DAGMA
model_hd = DagGenGRUg4s2v1(
    num_nodes=N, in_feats_dim=D, out_feats_dim=F,
    hidden_dim=hidden_dim, num_layers=1, num_heads=heads,
    feats_layers=1, dist_adj=dist_adj, agg_feats='ori',
    node_norm=False, use_norm=False,
    use_dagma=True, use_lib_dagma=False,  # فقط نسخه دستی
    dagma_mu=1.0, dagma_alpha=1.0, dagma_steps=2
)
model_hd.eval()
with torch.no_grad():
    graphs_hd, reconst_hd = model_hd(x_with_pre, gen_graph_only=False)

print("Hand‐written DAGMA → graphs:", graphs_hd.shape)
print("Hand‐written DAGMA → reconst:", reconst_hd.shape)


Hand‐written DAGMA → graphs: torch.Size([2, 1, 2, 3, 3])
Hand‐written DAGMA → reconst: torch.Size([2, 1, 3, 1])


In [4]:
from nns.case.dag_gen_gru import DagGenGRUg4s2v1

# مدل با پروجکتور دستی DAGMA
model_hd = DagGenGRUg4s2v1(
    num_nodes=N, in_feats_dim=D, out_feats_dim=F,
    hidden_dim=hidden_dim, num_layers=1, num_heads=heads,
    feats_layers=1, dist_adj=dist_adj, agg_feats='ori',
    node_norm=False, use_norm=False,
    use_dagma=True, use_lib_dagma=True,  # نسخه کتابخوانه ای
    dagma_mu=1.0, dagma_alpha=1.0, dagma_steps=2
)
model_hd.eval()
with torch.no_grad():
    graphs_hd, reconst_hd = model_hd(x_with_pre, gen_graph_only=False)

print("Hand‐written DAGMA → graphs:", graphs_hd.shape)
print("Hand‐written DAGMA → reconst:", reconst_hd.shape)


Hand‐written DAGMA → graphs: torch.Size([2, 1, 2, 3, 3])
Hand‐written DAGMA → reconst: torch.Size([2, 1, 3, 1])


In [6]:
graphs, _ = model_hd(x_with_pre, gen_graph_only=True)
print("graphs shape:", graphs.shape)  # Expect (T, B, 2, N, N)


graphs shape: torch.Size([2, 1, 2, 3, 3])


In [14]:
from nns.case.pred_blocks import CascadeGCNv2d4

pred_model = CascadeGCNv2d4(
    num_nodes=N,        # تعداد گره‌های گراف (مثلاً 3)
    input_dim=D,        # بعد ویژگی ورودی در هر گره (مثلاً 5)
    output_dim=F,       # بعد خروجی (مثلاً 1 برای سرعت/جریان)
    hidden_dim=16,      # سایز فضای نهان داخل GCN
    dist_adj=dist_adj,  # ماتریس فاصله ثابت (برای اجزای گراف پیش‌فرض)
    num_layers=2,       # تعداد لایه‌های cascade در هر بلوک
    seq_len=T,          # طول توالی ورودی (مثلاً 2)
    horizon=12       # طول توالی پیش‌بینی (مثلاً 2 یا 12)
)


In [15]:
x_hist = x_with_pre[1:]

preds = pred_model(x_hist, graphs)
print("preds shape:", preds.shape)  # Expect (horizon, B, N, out_dim)


preds shape: torch.Size([12, 1, 3, 1])


In [16]:
print(preds)             # خود مقادیر
print(preds.min(), preds.max())


tensor([[[[ 0.0692],
          [ 0.6082],
          [ 0.7238]]],


        [[[ 0.3340],
          [-0.1947],
          [-0.8503]]],


        [[[-0.1890],
          [-0.0078],
          [-0.2325]]],


        [[[-1.9260],
          [-0.8646],
          [ 0.3568]]],


        [[[-1.0610],
          [-0.8392],
          [-1.2524]]],


        [[[-0.2192],
          [-0.3442],
          [-0.5992]]],


        [[[ 0.2417],
          [ 0.5603],
          [ 0.6212]]],


        [[[-2.6781],
          [-2.2238],
          [-2.3295]]],


        [[[ 0.5733],
          [ 2.0626],
          [ 2.4193]]],


        [[[ 0.7970],
          [ 0.3763],
          [ 0.0873]]],


        [[[-0.0069],
          [ 0.0625],
          [ 0.3428]]],


        [[[-0.2959],
          [-0.0104],
          [-0.1796]]]], grad_fn=<PermuteBackward0>)
tensor(-2.6781, grad_fn=<MinBackward1>) tensor(2.4193, grad_fn=<MaxBackward1>)
